In [354]:
import requests
from web3 import Web3
import time
import pandas as pd

In [355]:
url = "https://api.chainbase.online/v1/dw/query"
headers = {
    "x-api-key": "..."
}
web3_arbitrum = Web3(Web3.HTTPProvider('https://arbitrum-one.public.blastapi.io'))
web3_bsc = Web3(Web3.HTTPProvider('https://bsc-dataseed2.defibit.io'))
web3_ethereum = Web3(Web3.HTTPProvider('https://eth.llamarpc.com'))
web3_polygon = Web3(Web3.HTTPProvider('https://polygon.llamarpc.com'))

In [356]:
# Arbitrum
arbitrum_chainid = '42161'
arbitrum_usdt = '0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9'
arbitrum_usdc = '0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8'
arbitrum_gasprice = str(web3_arbitrum.eth.gas_price/10 ** 9)
arbitrum = [arbitrum_chainid, arbitrum_usdt, arbitrum_usdc, arbitrum_gasprice]

# BNB Chain
bsc_chainid = '56'
bsc_usdt = '0x2B90E061a517dB2BbD7E39Ef7F733Fd234B494CA'
bsc_usdc = '0x8ac76a51cc950d9822d68b83fe1ad97b32cd580d'
bsc_gasprice = str(web3_bsc.eth.gas_price/10 ** 9)
bsc = [bsc_chainid, bsc_usdt, bsc_usdc, bsc_gasprice]

# Ethereum
ethereum_chainid = '1'
ethereum_usdt = '0xdac17f958d2ee523a2206206994597c13d831ec7'
ethereum_usdc = '0x7ea2be2df7ba6e54b1a9c70676f668455e329d29'
ethereum_gasprice = str(web3_ethereum.eth.gas_price/10 ** 9)
ethereum = [ethereum_chainid, ethereum_usdt, ethereum_usdc, ethereum_gasprice]

# Polygon
polygon_chainid = '137'
polygon_usdt = '0xc2132D05D31c914a87C6611C10748AEb04B58e8F'
polygon_usdc = '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174'
polygon_gasprice = str(web3_polygon.eth.gas_price/10 ** 9)
polygon = [polygon_chainid, polygon_usdt, polygon_usdc, polygon_gasprice]

# Merge
all_chain = [arbitrum, bsc, ethereum, polygon]

# url
#pre_url = "https://us_test-open-api.openocean.finance/v3"
#pre_url = "https://hk_open-api.openocean.finance/v3"
pre_url = "https://open-api.openocean.finance/v3"

In [357]:
df_responds_testing = pd.DataFrame(columns = ['test_round', 'chain', 'useTime'])
test_round = 0

In [358]:
def speed_test():
    for i in all_chain:
        quote = '/' + i[0] + '/quote?'
        url = pre_url + quote + 'inTokenAddress=' + i[1] + '&outTokenAddress=' + i[2] + '&amount=1&slippage=1&gasPrice=' + i[3]
        
        try:
            response = requests.get(url)
            response_json = response.json()
        except:
            tmp = {'test_round':test_round, 'chain':i[0], 'useTime':20}
            df_responds_testing.loc[len(df_responds_testing)]=tmp
            time.sleep(1)
            continue
        else:
            # tmp = {'test_round':test_round, 'chain':i[0], 'useTime':response['data']['useTime']}
            tmp = {'test_round':test_round, 'chain':i[0], 'useTime':response.elapsed.total_seconds()}
            
            df_responds_testing.loc[len(df_responds_testing)]=tmp
            time.sleep(1)

In [359]:
count = 10
while count > 0:
    speed_test()
    test_round += 1
    count -= 1

In [360]:
df_result = pd.DataFrame(columns = ['chain', 'avgTime', 'round'])

for i in all_chain:
    avgTime = df_responds_testing[df_responds_testing['chain'] == i[0]]['useTime'].mean()
    tmp = {'chain': i[0], 'avgTime': avgTime, 'round':test_round}
    df_result.loc[len(df_result)] = tmp

In [361]:
# 总体交易相应时间数据
df_result

,chain,avgTime,round
0,42161,3.777819,10
1,56,6.688424,10
2,1,16.571787,10
3,137,1.151734,10


In [362]:
# 响应时间过长的交易数据
df_responds_testing[df_responds_testing['useTime'] > 3]

,test_round,chain,useTime
1,0,56,10.861012
2,0,1,79.130859
5,1,56,7.652600
9,2,56,5.625637
10,2,1,4.123428
16,4,42161,32.433065
17,4,56,3.215070
25,6,56,9.792275
26,6,1,73.156626
29,7,56,5.855248
